In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import os
from datetime import datetime as dttm
import datetime
import json

In [114]:
# 용량 줄이기
# 데이터 범위 체크 후, 최적화된 타입 적용
def reduce_mem_usage(df):
    
    # 현재 메모리 사용량
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터 프레임의 메모리 사용량 : {:.2f} MB'.format(start_mem))
    
    
    # 컬럼 하나씩 확인 후 적용
    for col in tqdm(df.columns[1:]):
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            
    # 타입변경 후 메모리 사용량        
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 메모리 사용량 : {:.2f} MB'.format(end_mem))
    print('감소율 : {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [115]:
# Datetime 변환
def df_astype(df):
    reduce_mem_usage(df)
    df['STARTTIMESTAMP'] = pd.to_datetime(df['STARTTIMESTAMP'])
    df['ENDTIMESTAMP'] = pd.to_datetime(df['ENDTIMESTAMP'])

In [116]:
def day_split(file_name, year, month, save_encoding='cp949'):
    
    path = './전처리/wi-fi/path/'
            
    # csv 파일 읽어오기
    try:
        main_df = pd.read_csv(f'{path}{file_name}', encoding= 'cp949')
    except:
        main_df = pd.read_csv(f'{path}{file_name}', encoding= 'utf-8')

    # 타입 변환
    df_astype(main_df)
    
    # 다음 이동지가 있는 것만 찾기
    main_df = main_df[main_df['Next_MAC'] >= 0]
    
    # 폴더 생성
    try:
        directory = f'{path}{year}년_특정월_일별/{month}월/'
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: 폴더 생성불가 :' +  directory)
    
    # 시간대별 csv 파일생성
    print('''
    
    csv파일 저장 시작
    
    ''')
    
    
    days_30_month_list = [4,6,9,11]

    if month == 2:
        last_day = 28

        if year%4 == 0:
            last_day = 29

    elif month in days_30_month_list:
        last_day = 30
    else:
        last_day = 31


    for day in tqdm(range(1, last_day+1)):
        if day != last_day:
            sub_df = main_df[(main_df['STARTTIMESTAMP'] >= dttm.strptime(f'{year}-{month}-{day} 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S')) &
                    (main_df['STARTTIMESTAMP'] < dttm.strptime(f'{year}-{month}-{day+1} 00:00:00',
                                                          '%Y-%m-%d %H:%M:%S'))]
        elif ((month == 12) and (day == last_day)):
            sub_df = main_df[(main_df['STARTTIMESTAMP'] >= dttm.strptime(f'{year}-{month}-{day} 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S')) &
                    (main_df['STARTTIMESTAMP'] < dttm.strptime(f'{year+1}-1-1 00:00:00',
                                                          '%Y-%m-%d %H:%M:%S'))]
        else:
            sub_df = main_df[(main_df['STARTTIMESTAMP'] >= dttm.strptime(f'{year}-{month}-{day} 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S')) &
                    (main_df['STARTTIMESTAMP'] < dttm.strptime(f'{year}-{month+1}-1 00:00:00',
                                                          '%Y-%m-%d %H:%M:%S'))]    
    
    
        sub_df.to_csv(f'{directory}{year}년_{month}월_{day}일.csv', encoding=save_encoding, index=False)
    print('done')

In [117]:
day_split('presence_path_2022.csv', 2022, 7)

데이터 프레임의 메모리 사용량 : 1765.12 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 826.17 MB
감소율 : 53.2%

    
    csv파일 저장 시작
    
    


  0%|          | 0/31 [00:00<?, ?it/s]

done


In [122]:
def MAC_addr_split(folder, year, save_encoding='cp949'):
    
    path = f'./전처리/wi-fi/path/{folder}/'

    li = []

    # 디렉토리 내에서 디렉토리만 추출
    for file in os.listdir(path):
        X = os.path.join(path, file)
        if os.path.isdir(X):
            li.append(file)


    # 해당 디렉토리에서 csv 파일만 들고옴
    for f in li:
        csv_path = path + f +'/'
        fileEx = '.csv'
        time_list = [file for file in os.listdir(csv_path) if file.endswith(fileEx)]
    
    
    # 시간대별
    for n in tqdm(time_list):
        csv_file = csv_path + n
        folder = n.split('.')[0]
        
        # 폴더 생성
        try:
            directory = f'{csv_path}{folder}/'
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: 폴더 생성불가 : ' +  directory)
        
        # csv 파일 읽어오기
        try:
            df = pd.read_csv(f'{csv_file}', encoding='cp949')
        except:
            df = pd.read_csv(f'{csv_file}', encoding='utf-8')
        
        
        # Wi-Fi 개소 list
        df_mac_li = list(set(df['MAC_add']))

        # Wi-Fi 개소별 나누기
        for i in df_mac_li:
            small_df = df[df['MAC_add'] == i]
            small_df.to_csv(f'{directory}{folder}_{i}.csv', encoding=save_encoding, index=False)

In [123]:
MAC_addr_split('2022년_특정월_일별', 2022)

  0%|          | 0/31 [00:00<?, ?it/s]

In [126]:
def Path_counting(folder, year, save_encoding='cp949'):
    
    path = f'./전처리/wi-fi/path/{folder}/'

    month_li = []

    # 디렉토리 내에서 디렉토리만 추출
    for file in os.listdir(path):
        X = os.path.join(path, file)
        if os.path.isdir(X):
            month_li.append(file)

    for mon in month_li:
        mon_path = path + mon + '/'


        day_li = []

        # 디렉토리 내에서 디렉토리만 추출
        for file in os.listdir(mon_path):
            X = os.path.join(mon_path, file)
            if os.path.isdir(X):
                day_li.append(file)


        print('전체 진행상황')
        for n in tqdm(day_li):
            day_path = mon_path + n +'/'
            wifi_csv_list = os.listdir(day_path)

            try:
                directory = f'./전처리/wi-fi/path/count/{folder}/{mon}/{n}/'
                if not os.path.exists(directory):
                    os.makedirs(directory)
            except OSError:
                print ('Error: 폴더 생성불가 : ' +  directory)
            print('-----'*20)
            print(n)
            for i in tqdm(wifi_csv_list):
                
                # csv 파일 읽어오기
                try:
                    df = pd.read_csv(day_path + i , encoding='cp949')
                except:
                    df = pd.read_csv(day_path + i , encoding='utf-8')
                
                
                df = df[['MAC_add','Next_MAC']]
                df = pd.DataFrame(df.groupby(['MAC_add','Next_MAC']).value_counts(), columns=['count'])
                df.to_csv(f'{directory}{i}', encoding=save_encoding)

In [127]:
Path_counting('2022년_특정월_일별', 2022)

전체 진행상황


  0%|          | 0/31 [00:00<?, ?it/s]

----------------------------------------------------------------------------------------------------
2022년_7월_10일


  0%|          | 0/69 [00:00<?, ?it/s]

----------------------------------------------------------------------------------------------------
2022년_7월_11일


...

  0%|          | 0/69 [00:00<?, ?it/s]

----------------------------------------------------------------------------------------------------
2022년_7월_8일


  0%|          | 0/69 [00:00<?, ?it/s]

----------------------------------------------------------------------------------------------------
2022년_7월_9일


  0%|          | 0/69 [00:00<?, ?it/s]

In [128]:
# 위경도 추가
def lon_lat_append(folder, sace_encoding = 'cp949', ):
    
    # 폴더 생성 경로
    fol = './전처리/wi-fi/path/'
    
    # MAC 코드화 딕셔너리 불러오기
    with open('mac_code.json', 'r') as f :
        mac_code = json.load(f, encoding='utf-8')
    
    # key값과 values값 위치 변경
    mac_code = dict(zip(mac_code.values(),mac_code.keys()))
        
    # 위경도 불러오기
    wk = pd.read_csv('./전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])
    
    
    path = f'./전처리/wi-fi/path/count/{folder}/'
    
    month_li = []

    # 디렉토리 내에서 디렉토리만 추출
    for file in os.listdir(path):
        X = os.path.join(path, file)
        if os.path.isdir(X):
            month_li.append(file)

    for mon in month_li:
        mon_path = path + mon + '/'


        day_li = []

        # 디렉토리 내에서 디렉토리만 추출
        for file in os.listdir(mon_path):
            X = os.path.join(mon_path, file)
            if os.path.isdir(X):
                day_li.append(file)  
        
        print('전체 진행상황')
        for d in tqdm(day_li):
            day_path = mon_path + d +'/'
            wifi_csv_list = os.listdir(day_path)
    


            # 병합을 위한 변수
            cnct_df = None

            # csv 파일명으로 변수 생성 
            for n in tqdm(wifi_csv_list):

                # csv 파일 읽어오기
                try:
                    df = pd.read_csv(f'{day_path}{n}', encoding='cp949')
                except:
                    df = pd.read_csv(f'{day_path}{n}', encoding='utf-8')


                # 위경도 컬럼 추가 
                df['main_X'] = None
                df['main_Y'] = None
                df['next_X'] = None
                df['next_Y'] = None

                for a in range(len(df)):

                    # MAC 주소 코드화 해제
                    df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
                    df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]

                    # Main MAC 주소 위경도 추가 (기준 MAC)
                    df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
                    df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']

                    # 다음 이동경로 MAC 주소 위경도 추가
                    b = df['Next_MAC'][a]
                    df['next_X'][a] = wk.loc[b]['경도']
                    df['next_Y'][a] = wk.loc[b]['위도']

                # 폴더 생성
                try:
                    directory = fol + '위경도/count_' + folder + '/' + mon + '/' + d
                    if not os.path.exists(directory):
                        os.makedirs(directory)
                except OSError:
                    print ('Error: 폴더 생성불가 : ' +  directory)

                    
                # 생성한 폴더에 저장
                df.to_csv(f'{directory}/{n}', encoding=sace_encoding, index=False)

                
                # 데이터프레임 병합
                if n == wifi_csv_list[0]:
                    cnct_df = df
                    continue
                cnct_df = pd.concat([cnct_df, df])

            # 병합 데이터프레임 저장
            cnct_df.to_csv(f'{directory}/{d}_all.csv', encoding=sace_encoding, index=False)

In [129]:
lon_lat_append('2022년_특정월_일별')

전체 진행상황


  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/69 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17864\1566877349.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#